---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [ ]:
#import os
#os.chdir('/python')

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Hypothesis Testing
 

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

In [ ]:
House=pd.read_csv('City_Zhvi_AllHomes.csv')
gdp=pd.read_excel('gdplev.xls')

In [ ]:
town=pd.read_table('university_towns.txt')
town.rename(columns={"Alabama[edit]":'place'},inplace=True)


In [ ]:
gdp=pd.read_excel('gdplev.xls')

In [ ]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [ ]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    with open("university_towns.txt") as f:
        
        townslist = f.readlines()
        townslist = [x.rstrip() for x in townslist]
        townslist2 = list()
        for i in townslist:
            if i[-6:] == '[edit]':
                temp_state = i[:-6]
            elif '(' in i:
                town = i[:i.index('(') - 1]
                townslist2.append([temp_state, town])
            else:
                town = i
                townslist2.append([temp_state, town])

        collegedf = pd.DataFrame(townslist2, columns=['State','RegionName'])


        df1 = collegedf.convert_objects(convert_numeric=True)
    return df1
get_list_of_university_towns()

In [ ]:
s=gdp.iloc[4:]
s.rename(columns={"Unnamed: 4":'YEAR+Q',"Unnamed: 5":'GDP in billions of current dollars',"Unnamed: 6":'GDP in billions of chained 2009 dollars'},inplace=True) #unamed 5 unamed 6 unamed 4
s.reset_index(inplace=True)
s.drop('index',axis=1,inplace=True)
s.drop(['Current-Dollar and "Real" Gross Domestic Product','Unnamed: 1','Unnamed: 2','Unnamed: 3'],axis=1,inplace=True)
s=s[['YEAR+Q','GDP in billions of current dollars','GDP in billions of chained 2009 dollars']]
s=s.loc[215:]
s.reset_index(inplace=True)
s.drop('index',axis=1,inplace=True)
s['GDP in billions of current dollars'].iloc[10]
s['Rate']=s['GDP in billions of chained 2009 dollars'].pct_change()
s = s.convert_objects(convert_numeric=True)

In [ ]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    i=2
    a=[]
    while i < (s.shape[0]):
        if s.iloc[i-1]['Rate']<0 and  s.iloc[i]['Rate']<0: 
            a.append(s.iloc[i-1]['YEAR+Q']) 
        i+=1
    return a[0]

get_recession_start()

In [ ]:
def get_recession_end():
    
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    i=2
    a=[]
    s1=s[29:41]
    while i < (s1.shape[0]):
        if s1.iloc[i-1]['Rate']>0 and  s1.iloc[i]['Rate']>0: 
            a.append(s1.iloc[i-1]['YEAR+Q'])  
        i+=1
    return a[2]
get_recession_end()

In [ ]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    s1=s.iloc[34:40]
    s1[s1['GDP in billions of chained 2009 dollars']==s1['GDP in billions of chained 2009 dollars'].min()]['YEAR+Q']
    return s1.loc[37]['YEAR+Q']
get_recession_bottom()

In [ ]:
def convert_housing_data_to_quarters():
    
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    a=pd.Series(np.arange(2000,2017)).map(str)
    b=pd.Series(np.arange(1,13)).map(str)
    c=pd.Series(np.arange(1,5)).map(str)
    c=c.map(lambda x:"q"+x)
    b=b.map(lambda x:"0"+x)
    #House.drop()
    b[9:12]=["10","11","12"]
    i=0
    #j=0
    df=pd.DataFrame()
    while(i<17):
        if(i!=16):
            j=0
            while (j<4):
                if(j==0):
                    df[a[i]+c[j]]=House[[a[i]+"-"+b[0],a[i]+"-"+b[1],a[i]+"-"+b[2]]].mean(axis=1)
                elif(j==1):
                    df[a[i]+c[j]]=House[[a[i]+"-"+b[3],a[i]+"-"+b[4],a[i]+"-"+b[5]]].mean(axis=1)   
                elif(j==2):
                    df[a[i]+c[j]]=House[[a[i]+"-"+b[6],a[i]+"-"+b[7],a[i]+"-"+b[8]]].mean(axis=1) 
                else:
                    df[a[i]+c[j]]=House[[a[i]+"-"+b[9],a[i]+"-"+b[10],a[i]+"-"+b[11]]].mean(axis=1) 
                j+=1
        else:
            j=0
            while (j<3):
                if(j==0):
                    df[a[i]+c[j]]=House[[a[i]+"-"+b[0],a[i]+"-"+b[1],a[i]+"-"+b[2]]].mean(axis=1)
                elif(j==1):
                    df[a[i]+c[j]]=House[[a[i]+"-"+b[3],a[i]+"-"+b[4],a[i]+"-"+b[5]]].mean(axis=1)   
                else:
                    df[a[i]+c[j]]=House[[a[i]+"-"+b[6],a[i]+"-"+b[7]]].mean(axis=1) 
                j+=1
        i+=1
    df['State']=House['State'].map(states)
    df['RegionName']=House['RegionName']
    df.set_index(['State','RegionName'],inplace=True)
    return df
convert_housing_data_to_quarters()

In [ ]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    towns = get_list_of_university_towns()
    startdate = get_recession_start()
    bottomdate = get_recession_bottom()
    houses = convert_housing_data_to_quarters()
    houses = houses.reset_index()
    houses['recession_diff'] = houses[startdate] - houses[bottomdate]
    unitowns=pd.merge(houses,towns,how='inner',on=['State','RegionName'])
    l=unitowns.index
    not_unitowns=houses[~houses.index.isin(l)]
    t,p=ttest_ind(unitowns['recession_diff'].dropna(), not_unitowns['recession_diff'].dropna())
    difference=True if p<0.01 else False
    better = "university town" if unitowns['recession_diff'].mean() < not_unitowns['recession_diff'].mean() else "non-university town"
    return difference,p,better
run_ttest()